In [ ]:
import sys
sys.path.append('../')
from delta_debugging.DD_mod import DDMods
import subprocess

In [ ]:
class TestDD(DDMods):
    def __init__(self, fuzzer_path):
        DDMods.__init__(self)
        self.debug_dd = 0
        self.verbose = 0
        self.fuzzer_path = fuzzer_path
    def _test(self, deltas):
        # Reconstruct the binary input
        binary_data = deltas_to_bytes(deltas)
        print("Testing input of length {}...".format(len(binary_data)))
        if not binary_data:
            return self.PASS
        # Run the binary with the input
        result = self.test_seed(binary_data)
        return self.PASS if result else self.FAIL
    def test_seed(self, binary_input):
        """Run fuzzer with binary_input piped to stdin and return True if it doesn't crash."""
        try:
            result = subprocess.run(
                [self.fuzzer_path],
                input=binary_input,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                check=False
            )
            return result.returncode == 0
        except Exception as e:
            print("Error running fuzzer: {}".format(e))
            return False

In [ ]:
def bytes_to_deltas(b):
    return list(map(lambda i: (i, bytes(b[i])), range(len(b))))

In [ ]:
def deltas_to_bytes(deltas):
    return b"".join([x[1] for x in deltas])

## Min fail input

In [ ]:
crashing_input = "/src/clusterfuzz-testcase-minimized-server-6088352019251200"
valid_input = "/src/boringssl/fuzz/server_corpus/1037d8ef263682ea727da03710954173df7212ac"
fuzzer = "server"
mydd = TestDD(fuzzer_path=fuzzer)

In [9]:
mydd = TestDD()
string1 = '{"baz": 7, "baaa": [1, 2]}'
# string1 = '{"baaa": [1, 2]}'
test_input = string1
deltas = list(map(lambda x: (x, test_input[x]), range(len(test_input))))
c = mydd.ddmin(deltas)              # Invoke DDMIN
minimal = "".join([x[1] for x in c])

Testing case "          . 
Test passed: empty string
Testing case "{"baz": 7, "baaa": [1, 2]}. 
Test failed: baaa is a list.
Testing case "aaa": [1, 2]}. 
Test unresolved: Expecting value: line 1 column 1 (char 0)
Testing case "{"baz": 7, "b. 
Test unresolved: Unterminated string starting at: line 1 column 12 (char 11)
Testing case ": 7, "baaa": [1, 2]}. 
Test unresolved: Expecting value: line 1 column 1 (char 0)
Testing case "{"baz"baaa": [1, 2]}. 
Test unresolved: Expecting ':' delimiter: line 1 column 7 (char 6)
Testing case "{"baz": 7, "[1, 2]}. 
Test unresolved: Unterminated string starting at: line 1 column 12 (char 11)
Testing case "{"baz": 7, "baaa": . 
Test unresolved: Expecting value: line 1 column 20 (char 19)
Testing case "az": 7, "baaa": [1, 2]}. 
Test unresolved: Expecting value: line 1 column 1 (char 0)
Testing case "{"b: 7, "baaa": [1, 2]}. 
Test unresolved: Expecting ':' delimiter: line 1 column 10 (char 9)
Testing case "{"baz", "baaa": [1, 2]}. 
Test unresolved: Expec

In [10]:
print('Found minimal test case: "{}"'.format(minimal))

Found minimal test case: "{"":7,"":[]}"
